In [1]:
import pandas as pd
from config import CONFIGS
import os
import re
import os
import json
from typing import Tuple

# from utils.processing_functions import load_file_local_first, save_file_local_first
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer


ENVIRONMENT = os.environ.get("ENVIRONMENT", "dev")
S3_SCRAPER_BUCKET = CONFIGS["s3_scraper_bucket"]
GAME_CONFIGS = CONFIGS["games"]
RATINGS_CONFIGS = CONFIGS["ratings"]
IS_LOCAL = True if os.environ.get("IS_LOCAL", "False").lower() == "true" else False

IS_LOCAL: True

Checking for local config file and evaluating for updates from S3.
Loading config from local


In [2]:
def get_major_and_all_components(component_type:str, df:pd.DataFrame) -> Tuple[list, list]:

    # Extract unique components from components
    unique_positive_components = set()
    for components in df[f'{component_type}_Components']:
        unique_positive_components.update(components)
    major_components = [x for x in list(unique_positive_components) if x != ''] 
    # sort major components by number of words in the entry, highest to lowest
    major_components = sorted(major_components, key=lambda x: len(x.split()), reverse=True)

    # Extract unique elements from both Positive_Components and Positive_Sentences
    unique_sentence_components = set()  # Start with Positive_Components
    for sentences in df[f'{component_type}_Sentences']:
        unique_sentence_components.update(sentences)
    sentence_components = [x for x in list(unique_sentence_components) if x != ''] 
    sentence_components = sorted(sentence_components, key=lambda x: len(x.split()), reverse=True)

    # Extract unique elements from both Positive_Components and Positive_Sentences
    unique_all_components = set(unique_positive_components)  # Start with Positive_Components
    for sentences in df[f'{component_type}_Sentences']:
        unique_all_components.update(sentences)
    all_components = [x for x in list(unique_all_components) if x != ''] 
    all_components = sorted(all_components, key=lambda x: len(x.split()), reverse=True)


    return all_components, major_components, sentence_components

# Weaviate create attribute embeddings

In [ ]:
import weaviate
import weaviate.classes as wvc
from pydantic import BaseModel, ConfigDict
from weaviate.classes.config import Configure
from weaviate.classes.query import Filter
from weaviate.classes.query import MetadataQuery
from weaviate.util import generate_uuid5

class WeaviateClient(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    weaviate_client: weaviate.client = None
    collection: weaviate.collections.Collection = None

    def model_post_init(self, __context):
        self.weaviate_client = self.connect_weaviate_client_docker()

    def connect_weaviate_client_docker(self) -> weaviate.client:
        if not IS_LOCAL:
            client = weaviate.connect_to_local(
                host="127.0.0.1",
                port=8081,
                grpc_port=50051,
                headers={
                    "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"],
                },
            )
            return client

        return weaviate.connect_to_local(
            port=8081,
            headers={
                "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"],
            },
        )
    
    def find_near_objects(self, collection_name, uuid, limit:int =20):
        self.collection = self.weaviate_client.collections.get(collection_name)
        response = self.collection.query.near_object(
            near_object=uuid,
            limit=limit,
            return_metadata=MetadataQuery(distance=True),
        )
        return response.objects
    
    def close_client(self):
        self.weaviate_client.close()
    
    def add_similarity_collection_item(self, item:pd.Series, collection_name: str = "similarity_collection") -> None:

        self.collection = self.weaviate_client.collections.get(collection_name)

        print(f"Adding data for game {item["BGGId"]}")

        game_object = {
            "bggid": str(item["BGGId"]),
            "name": str(item["Name"]).lower(),
            # "description": str(item["Description"]).lower(),
            "about": str(item["About"]).lower(),
            "positive": str(item["Positive"]).lower(),
            "negative": str(item["Negative"]).lower(),
        }
        
        uuid = generate_uuid5(game_object)

        if self.collection.data.exists(uuid):
            return
        else:
            self.collection.data.insert(properties=game_object, uuid=uuid)

        return uuid

    def create_similarity_collection(self, collection_name: str = "similarity_collection") -> None:

        if self.weaviate_client.collections.exists(collection_name):
            print("Collection already exists for this block")
            self.weaviate_client.collections.delete(collection_name)
            print("Deleted and recreating collection")
            return

        self.weaviate_client.collections.create(
            name=collection_name,
            vectorizer_config=[
                Configure.NamedVectors.text2vec_transformers(
                    name="title_vector",
                    source_properties=["title"],
                )
            ],
            properties=[
                wvc.config.Property(
                    name="bggid",
                    data_type=wvc.config.DataType.TEXT,
                    skip_vectorization=True,
                    vectorize_property_name=False,
                ),
                wvc.config.Property(
                    name="name",
                    data_type=wvc.config.DataType.TEXT,
                    skip_vectorization=True,
                    vectorize_property_name=False,
                ),
                # wvc.config.Property(
                #     name="description", data_type=wvc.config.DataType.TEXT
                # ),
                wvc.config.Property(name="about", data_type=wvc.config.DataType.TEXT),
                wvc.config.Property(
                    name="positive", data_type=wvc.config.DataType.TEXT
                ),
                wvc.config.Property(
                    name="negative", data_type=wvc.config.DataType.TEXT
                ),
            ],
        )

    def add_attributes_collection_item(self, attribute:str, collection_name: str = "game_attributes") -> None:

        self.collection = self.weaviate_client.collections.get(collection_name)

        print(f"Adding data for attribute {attribute}")

        attribute_object = {
            "attribute_name": attribute,
            "attribute": attribute,
        }
        
        uuid = generate_uuid5(attribute_object)

        if self.collection.data.exists(uuid):
            return uuid
        else:
            self.collection.data.insert(properties=attribute_object, uuid=uuid)

        return uuid
    
    def create_attributes_collection(self, collection_name: str = "game_attributes") -> None:

        if self.weaviate_client.collections.exists(collection_name):
            print("Collection already exists for this block")
            self.weaviate_client.collections.delete(collection_name)
            print("Deleted and recreating collection")
            return

        self.weaviate_client.collections.create(
            name=collection_name,
            vectorizer_config=[
                Configure.NamedVectors.text2vec_transformers(
                    name="title_vector",
                    source_properties=["title"],
                )
            ],
            properties=[
                wvc.config.Property(
                    name="attribute_name",
                    data_type=wvc.config.DataType.TEXT,
                    skip_vectorization=True,
                    vectorize_property_name=False,
                ),
                wvc.config.Property(
                    name="attribute", data_type=wvc.config.DataType.TEXT, vectorize_property_name=False
                ),
            ],
        )

    def close_client(self):
        self.weaviate_client.close()


weaviate_client = WeaviateClient()

In [ ]:
df = pd.read_pickle("top_1000_cleaned_rag.pkl")
df.head()

### Create Positives Storage

In [ ]:
all_positives, major_positives, sentence_positives = get_major_and_all_components("Positive", df)

In [ ]:
weaviate_client.create_attributes_collection(collection_name="positives")

positive_attributes_store = {}

for item in all_positives:
    uuid = weaviate_client.add_attributes_collection_item(item, collection_name="positives")
    positive_attributes_store[item] = uuid

In [ ]:
match_storage = {}
total_entries = len(all_positives)
entries_completed = 0


for major_component in all_positives:
    uuid = positive_attributes_store[major_component]
    pos_similars = weaviate_client.find_near_objects(collection_name="positives", uuid=uuid, limit=2000)
    matches_without_major_component = {x.properties['attribute_name']:x.metadata.distance for x in pos_similars if x.metadata.distance <= .40 and not x.properties['attribute_name'].startswith(major_component)}
    match_storage[major_component] = matches_without_major_component
    # report back every 100 entries completed
    entries_completed += 1
    if entries_completed % 100 == 0:
        print(f"Completed {entries_completed} of {total_entries}")
    

In [ ]:
with open('positive_matches.json', 'w') as f:
    json.dump(match_storage, f)

### Create Negatives Storage

In [ ]:
all_negatives, major_negatives, sentence_negatives = get_major_and_all_components("Negative", df)

In [ ]:
weaviate_client.create_attributes_collection(collection_name="negatives")

negative_attributes_store = {}

for item in all_negatives:
    uuid = weaviate_client.add_attributes_collection_item(item, collection_name="negatives")
    negative_attributes_store[item] = uuid

In [ ]:
match_storage = {}
total_entries = len(all_negatives)
entries_completed = 0

for major_component in all_negatives:
    uuid = negative_attributes_store[major_component]
    pos_similars = weaviate_client.find_near_objects(collection_name="negatives", uuid=uuid, limit=2000)
    matches_without_major_component = {x.properties['attribute_name']:x.metadata.distance for x in pos_similars if x.metadata.distance <= .40 and not x.properties['attribute_name'].startswith(major_component)}
    match_storage[major_component] = matches_without_major_component
    entries_completed += 1
    if entries_completed % 100 == 0:
        print(f"Completed {entries_completed} of {total_entries}")
    

In [ ]:
with open('negative_matches.json', 'w') as f:
    json.dump(match_storage, f)

### Clean up positives and negatives into vectors

In [3]:
df = pd.read_pickle("top_1000_cleaned_rag.pkl")
df.head()

,BGGId,Description,About,Positive_Components,Negative_Components,Positive_Sentences,Negative_Sentences
0,224517,Brass: Birmingham is an economic strategy game...,This game is a strategic economic simulation s...,"[strategic depth, excellent design, player int...","[steep learning curve, fiddly mechanics, varia...",[strategic depth games complexity multitude de...,[steep learning curve players find rules diffi...
1,161936,Pandemic Legacy is a co-operative campaign gam...,This game is a cooperative board game where pl...,"[engaging storyline, cooperative gameplay, rep...","[complexity, time commitment, group dependency...",[engaging storyline evolving narrative keeps p...,[complexity players find numerous choices rule...
2,174430,Gloomhaven is a game of Euro-inspired tactica...,This game is a cooperative tactical adventure ...,"[content variety, replayability, tactical dept...","[setup time, complexity, pacing issues, high c...",[content variety immense amount content ensuri...,[setup time many reviews mention game requires...
3,342942,"In Ark Nova, you will plan and design a modern...",This game is a strategic board game where play...,"[engaging theme, unique mechanics, high replay...","[lengthy gameplay, luck factor, complexity, an...",[engaging theme zoo management theme appealing...,[lengthy gameplay many reviews highlight game ...
4,363622,Castles of Burgundy is a legendary Board Game ...,This game is a strategic board game that revol...,"[high production quality, engaging mechanics, ...","[complexity new players, setup time, potential...",[high production quality game boasts exception...,[complexity new players reviews mention game c...


In [4]:
len(df)

1000

In [5]:
all_positives, major_positives, sentence_positives = get_major_and_all_components("Positive", df)
all_negatives, major_negatives, sentence_negatives = get_major_and_all_components("Negative", df)

len(all_positives), len(major_positives), len(sentence_positives), len(all_negatives), len(major_negatives), len(sentence_negatives)

(6636, 932, 5706, 6694, 1424, 5283)

In [6]:
keys_storage = {"positive":[all_positives, major_positives, sentence_positives], "negative":[all_negatives, major_negatives, sentence_negatives]}

In [7]:
attributes_store = {"positive":json.loads(open("positive_matches.json").read()),
                    "negative":json.loads(open("negative_matches.json").read())}

In [10]:
df_as_dict = df.to_dict(orient='records')

for game_entry in df_as_dict:
    print(f"Preparing BGG ID: {game_entry['BGGId']}")

    keyword_mood_bar = {x:"positive" for x in game_entry['Positive_Components']}
    keyword_mood_bar.update({x:"negative" for x in game_entry['Negative_Components']})
    
    for component in game_entry['Positive_Components'] + game_entry['Negative_Components']:
        game_entry[f"{keyword_mood_bar[component]} {component}"] = 1

    sentence_mood_bar = {x:"positive" for x in game_entry['Positive_Sentences']}
    sentence_mood_bar.update({x:"negative" for x in game_entry['Negative_Sentences']})

    for review_key, mood in keyword_mood_bar.items():

        attribute_store = attributes_store[mood]
        key_store = keys_storage[mood]
        this_items_matches = attribute_store[review_key]

        keyword_values = {}

        keywords_to_iterate = list(this_items_matches.keys())

        while(len(keywords_to_iterate)):
            key_or_phrase = keywords_to_iterate.pop(0)
            this_item_major_key = [x for x in key_store[1] if key_or_phrase.startswith(x)][0]
            if f"{mood} {this_item_major_key}" not in game_entry:
                game_entry[f"{mood} {this_item_major_key}"] = 1 - this_items_matches[key_or_phrase]
            
            for item in keywords_to_iterate:
                if item.startswith(this_item_major_key):
                    keywords_to_iterate.remove(item)

    for review_sentence, mood in sentence_mood_bar.items():
        attribute_store = attributes_store[mood]
        key_store = keys_storage[mood]

        current_major_key = [x for x in key_store[1] if review_sentence.startswith(x)][0]

        this_item_sentence_matches = attribute_store[review_sentence]
        sentence_matches_excluding_current_major_key = {x:y for x,y in this_item_sentence_matches.items() if not x.startswith(current_major_key)}

        while len(sentence_matches_excluding_current_major_key):
            # get the first dictionary item
            key_or_phrase = list(sentence_matches_excluding_current_major_key.keys())[0]
            # print(key_or_phrase)
            # print(sentence_matches_excluding_current_major_key[key_or_phrase])

            this_item_major_key = [x for x in key_store[1] if key_or_phrase.startswith(x)][0]
            # print(this_item_major_key)

            all_entries_starting_with_key = [x for x in sentence_matches_excluding_current_major_key if x.startswith(this_item_major_key)]

            ratings_for_this_item_major_key = 1 - sentence_matches_excluding_current_major_key[all_entries_starting_with_key[0]]
            # print(ratings_for_this_item_major_key)

            if f"{mood} {this_item_major_key}" not in game_entry:
                game_entry[f"{mood} {this_item_major_key}"] = ratings_for_this_item_major_key
            else:
                game_entry[f"{mood} {this_item_major_key}"] = max(game_entry[f"{mood} {this_item_major_key}"], ratings_for_this_item_major_key)
            
            # delete all things in sentence_matches_excluding_current_major_key that start with this_item_major_key
            for item in all_entries_starting_with_key:
                del sentence_matches_excluding_current_major_key[item]
            
            # print("\n")

Preparing BGG ID: 224517
Preparing BGG ID: 161936
Preparing BGG ID: 174430
Preparing BGG ID: 342942
Preparing BGG ID: 363622
Preparing BGG ID: 233078
Preparing BGG ID: 316554
Preparing BGG ID: 167791
Preparing BGG ID: 115746
Preparing BGG ID: 187645
Preparing BGG ID: 291457
Preparing BGG ID: 162886
Preparing BGG ID: 220308
Preparing BGG ID: 12333
Preparing BGG ID: 182028
Preparing BGG ID: 84876
Preparing BGG ID: 193738
Preparing BGG ID: 169786
Preparing BGG ID: 246900
Preparing BGG ID: 173346
Preparing BGG ID: 28720
Preparing BGG ID: 248562
Preparing BGG ID: 167355
Preparing BGG ID: 271320
Preparing BGG ID: 266507
Preparing BGG ID: 177736
Preparing BGG ID: 124361
Preparing BGG ID: 397598
Preparing BGG ID: 295770
Preparing BGG ID: 341169
Preparing BGG ID: 205637
Preparing BGG ID: 312484
Preparing BGG ID: 120677
Preparing BGG ID: 237182
Preparing BGG ID: 266192
Preparing BGG ID: 192135
Preparing BGG ID: 164928
Preparing BGG ID: 96848
Preparing BGG ID: 251247
Preparing BGG ID: 324856
Prep

In [11]:
compare_df = pd.DataFrame(df_as_dict).fillna(0)
compare_df.head()

,BGGId,Description,About,Positive_Components,Negative_Components,Positive_Sentences,Negative_Sentences,positive strategic depth,positive excellent design,positive player interaction,...,positive attractive artwork,negative mixed reception guilds,negative catch mechanics,negative activity vs,negative action limitations,negative difficulty levels,positive fast playtime,positive variety guilds,positive simple ruleset,positive theme artwork
0,224517,Brass: Birmingham is an economic strategy game...,This game is a strategic economic simulation s...,"[strategic depth, excellent design, player int...","[steep learning curve, fiddly mechanics, varia...",[strategic depth games complexity multitude de...,[steep learning curve players find rules diffi...,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,161936,Pandemic Legacy is a co-operative campaign gam...,This game is a cooperative board game where pl...,"[engaging storyline, cooperative gameplay, rep...","[complexity, time commitment, group dependency...",[engaging storyline evolving narrative keeps p...,[complexity players find numerous choices rule...,0.815851,0.000000,0.797206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,174430,Gloomhaven is a game of Euro-inspired tactica...,This game is a cooperative tactical adventure ...,"[content variety, replayability, tactical dept...","[setup time, complexity, pacing issues, high c...",[content variety immense amount content ensuri...,[setup time many reviews mention game requires...,0.864504,0.667041,0.669815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,342942,"In Ark Nova, you will plan and design a modern...",This game is a strategic board game where play...,"[engaging theme, unique mechanics, high replay...","[lengthy gameplay, luck factor, complexity, an...",[engaging theme zoo management theme appealing...,[lengthy gameplay many reviews highlight game ...,1.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,363622,Castles of Burgundy is a legendary Board Game ...,This game is a strategic board game that revol...,"[high production quality, engaging mechanics, ...","[complexity new players, setup time, potential...",[high production quality game boasts exception...,[complexity new players reviews mention game c...,0.780845,0.644698,0.827433,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
compare_df.to_pickle("top_1000_cleaned_rag_with_ratings_extrap.pkl")

In [ ]:
break